In [2]:
#Blatantly copied from https://github.com/onnx/tutorials/blob/master/tutorials/PytorchCaffe2MobileSqueezeNet.ipynb
# Some standard imports
import io
import numpy as np
import torch.onnx
%matplotlib inline

In [5]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import torch

def findFiles(path): return glob.glob(path)

print(findFiles('data/names/*.txt'))

import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

# Build the category_lines dictionary, a list of names per language
category_lines = {}
all_categories = []

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

for filename in findFiles('data/names/*.txt'): #TODO: Find data!
    print("got it!")
    category = filename.split('/')[-1].split('.')[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

[]
Slusarski


Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 13 to 25 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 26 to 38 
    0     0     0     0     0     0     0     0     0     1     0     0     0

Columns 39 to 51 
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 52 to 56 
    0     0     0     0     0
[torch.FloatTensor of size 1x57]

torch.Size([5, 1, 57])


In [ ]:
import torch

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

print(letterToTensor('J'))

print(lineToTensor('Jones').size())

In [ ]:
import torch.nn as nn
from torch.autograd import Variable

class MyLSTMLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MyLSTMLayer, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.forgetGate = nn.Linear(input_size + hidden_size, hidden_size)
        self.incorporatePositionGate = nn.Linear(input_size + hidden_size, hidden_size)
        self.incorporateValueGate = nn.Linear(input_size + hidden_size, hidden_size)
        self.hiddenValueGate = nn.Linear(hidden_size, hidden_size)
        
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()
        
    def forward_step(self, input, hidden, cell_state):
        combined = torch.cat((input, hidden), 1)
        
        f = self.sigmoid(self.forgetGate(combined))
        i = self.sigmoid(self.incorporatePositionGate(combined))
        C_new = self.tanh(self.incorporateValueGate(combined))
        
        cell_state = f * cell_state + i * C_new
        
        hidden = self.relu(self.hiddenValueGate(cell_state))
        
        return hidden, cell_state
    
    def forward(self, input):
        hidden, cell_state = self.initAll()
        
        outputs = []
        
        for i in range(input.size()[0]):
            hidden, cell_state = self.forward_step(input[i], hidden, cell_state)
            outputs.append(hidden)
        
        return torch.stack(outputs)

    def initHidden(self):
        return Variable(torch.zeros(1, self.hidden_size)).cuda()
    
    def initCellState(self):
        return Variable(torch.zeros(1, self.hidden_size)).cuda()
    
    def initAll(self):
        return self.initHidden(), self.initCellState()

class myLSTM(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size, layers):
        super(myLSTM, self).__init__()
        
        self.lstm_layers = nn.ModuleList()
        input_sizes = [input_size] + hidden_sizes[:-1] 
        
        for input_size, hidden_size in zip(input_sizes, hidden_sizes):
            self.lstm_layers.append(MyLSTMLayer(input_size, hidden_size))
        
        self.outputGate = nn.Linear(hidden_sizes[-1], output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input):
        hiddens = input
        
        for i, lstm_layer in enumerate(self.lstm_layers):
            hiddens = lstm_layer(hiddens)
        
        return self.softmax(self.outputGate(hiddens[-1]))
        
n_hidden = 128
lstm = myLSTM(n_letters, [n_hidden, n_hidden], n_categories, 2).cuda()

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init


def get_model(pretrained=False, **kwargs):
    """
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = myLSTM(**kwargs)
    if pretrained:
        model = train_all() #TODO: cpy this in!
torch_model = squeezenet1_1(True)
    return model

In [ ]:
# Get pretrained squeezenet model
torch_model = get_model(True)

In [ ]:
from torch.autograd import Variable
batch_size = 1    # just a random number

# Input to the model
x = Variable(torch.randn(batch_size, 3, 224, 224), requires_grad=True) #TODO: Real input

# Export the model
torch_out = torch.onnx._export(torch_model,             # model being run
                               x,                       # model input (or a tuple for multiple inputs)
                               "names.onnx",       # where to save the model (can be a file or file-like object)
                               export_params=True)      # store the trained parameter weights inside the model file


In [ ]:
# Export to mobile
from onnx_caffe2.backend import Caffe2Backend as c2

init_net, predict_net = c2.onnx_graph_to_caffe2_net(model.graph)
with open("names_init_net.pb", "wb") as f:
    f.write(init_net.SerializeToString())
with open("names_predict_net.pb", "wb") as f:
    f.write(predict_net.SerializeToString())
